In [1]:
%load_ext autoreload
%autoreload 2

import os
import pickle as pkl
import subprocess
import pandas as pd

import napari
from magicgui import magicgui

from multiplex_pipeline.core_selection.viewer_utils import display_saved_rois, save_rois_from_viewer, redo_cores_layer, redo_bbox_layer
from multiplex_pipeline.utils.utils import change_to_wsl_path, get_package_path, get_workstation_path, load_config
from multiplex_pipeline.im_utils import get_org_im_shape, prepare_rgb_image
from multiplex_pipeline.roi_utils import read_in_saved_rois, get_refined_rectangles, get_visual_rectangles, prepare_poly_df_for_saving, xywh_to_corners

## Read in config

In [2]:
# load config
# this step ensures that all starting parameters are loaded before they are used
config = load_config(namespace = globals())

Using workstation DESKTOP-S71S74E

IM_LEVEL: 6
MIN_AREA: 2000
MAX_AREA: 10000
MIN_IOU: 0.8
MIN_ST: 0.9
MIN_INT: 15
FRAME: 4


In [3]:
# change the config settings if necessary
#IM_LEVEL = 6
MAX_AREA = 14000

## Read in data

In [4]:
# pathway to the image for segmentation
im_path = r'R:\CellDive\BLCA-1B\BLCA-1B_Final\BLCA-1B_1.0.4_R000_DAPI__FINAL_F.ome.tif'

# output pathway - where to save corrected cores coordinates
save_path = r'R:\Wayne\BLCA\BLCA-1B_Analysis\cores.csv'

# ensure that the directory exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

In [5]:
# read in image data

# get the original image shape
org_im_shape = get_org_im_shape(im_path)

# prepare the image for display
im_rgb = prepare_rgb_image(im_path, req_level=IM_LEVEL)
im = im_rgb[:, :, 0]
im_rgb.shape

c:\Users\KMK280\AppData\Local\miniconda3\envs\napari-env\Lib\site-packages\zarr\creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)


(1072, 558, 3)

## Create a viewer with options

In [22]:
viewer = napari.Viewer()
viewer.add_image(im)

# add a red rectangle to frame the image
frame_rect = xywh_to_corners([0,0,im.shape[1],im.shape[0]], frame = 0)
viewer.add_shapes(frame_rect, edge_color='white', face_color = 'transparent', shape_type='rectangle', edge_width=2, name = 'frame')

# add a layer for the saved rois
display_saved_rois(viewer, IM_LEVEL = IM_LEVEL, save_path = save_path)


#########################################################################################
# add diplay saved button
@magicgui(auto_call=False, call_button="Display Saved Cores", result_widget=False)
def display_saved_rois_button(viewer: napari.Viewer):
    display_saved_rois(viewer, IM_LEVEL = IM_LEVEL, save_path = save_path)  

# Add widget to viewer
viewer.window.add_dock_widget(display_saved_rois_button, area='left')

#########################################################################################
# add a save button
@magicgui(auto_call=False, call_button="Save Cores")  
def save_button(viewer: napari.Viewer):
    save_rois_from_viewer(viewer, org_im_shape = org_im_shape, req_level = IM_LEVEL, save_path = save_path)

# Add widget to viewer
viewer.window.add_dock_widget(save_button, area='left')

## Get suggestions for core detection from the SAM2 model

Execute the cell below to get suggestions from SAM model about the cores position. 

In [31]:
# Path to the Python interpreter in the target Conda environment
sam_env = get_workstation_path(key="sam_env")

# Path to your script
script_path = change_to_wsl_path(os.path.join(get_package_path('multiplex_pipeline'),'core_selection','suggest_cores.py'))
im_wsl_path = change_to_wsl_path(im_path)

# Define the script arguments
output_path_suggested = os.path.join(os.getcwd(), 'masks.pkl')

optional_args = ['--output', change_to_wsl_path(output_path_suggested)]

# Combine the command
command = ['wsl', sam_env, '-u', script_path, im_wsl_path, str(IM_LEVEL)] + optional_args

# Execute the script
process = subprocess.Popen(
    command,
    stdout=subprocess.PIPE,  # Pipe the standard output
    stderr=subprocess.PIPE,  # Pipe the standard error
    text=True                # Capture output as text
)

# Display output in real time
while True:
    output = process.stdout.readline()
    if output == "" and process.poll() is not None:
        break
    if output:
        print(output, end="")  # Print each line as it becomes available

# Print any remaining errors
errors = process.stderr.read()
if errors:
    print(f"ERROR: {errors}")

# load the masks
masks = pkl.load(open(output_path_suggested, 'rb'))
print(f'Initial number of masks: {len(masks)}')

# remove the masks file
os.remove(output_path_suggested)

# refine masks
rect_list = get_refined_rectangles(masks, im = im_rgb[:,:,0], frame = FRAME, min_area = MIN_AREA, max_area = MAX_AREA, min_iou = MIN_IOU, min_stability = MIN_ST, min_int = MIN_INT)
print('Number of refined masks:', len(rect_list))

# prepare df
df = prepare_poly_df_for_saving(rect_list, poly_types = ['rectangle']*len(rect_list), org_im_shape = org_im_shape, req_level = IM_LEVEL)
rect_list = get_visual_rectangles(df, IM_LEVEL)
poly_list = [(x/(2**IM_LEVEL)).astype('int') for x in df.polygon_vertices.to_list()]

# add to napari
redo_cores_layer(viewer,rect_list,shape_type = df.poly_type.to_list())
redo_bbox_layer(viewer,rect_list,df['core_name'].tolist())

No model path provided. Using default model path: /mnt/d/data_analysis/2024_bladder/sam2
Input image: /mnt/r/CellDive/BLCA-1B/BLCA-1B_Final/BLCA-1B_1.0.4_R000_DAPI__FINAL_F.ome.tif
Results will be saved to: /mnt/d/multiplex_pipeline/notebooks/masks.pkl

Preparing RGB image for segmentation...
Segmenting image. It should take around 1 min. Started at 2025-09-12 11:05:04...
Saving masks...
Initial number of masks: 101
Number of refined masks: 49
